In [1]:
import numpy as np
import torch
import pickle
import os
import csv
import scipy.optimize as optim

from sklearn import preprocessing
from sklearn.metrics import roc_auc_score

In [ ]:
class LFR(self, X_s, X_ns, y_S, y_ns, k):
  def __init__():
    """Here we will instantiate all of our parameters.
    Learnable parameters: V (our prototype space), w (the weights associated with y_hat), and alpha (coefficients for distance)
    non-learnable parameters: M_nk, M_k """

    #TODO: will need to initialize & fill in dimensions accordingly 
    P = X_s.shape[1]

    #learnable parameters --> make sure to turn into torch.Parameters()!!
    self.v = torch.nn.Parameter(torch.rand(k, P)) #kxP
    self.w = torch.nn.Parameter(torch.rand(1, k)) #1xk
    self.alpha = torch.nn.Parameter(torch.rand(1, 2*P)) #1xP

    #unlearnable parameters
    
    #keep record of sensitive vs non-sensitive coefficients
    self.alpha_S = self.alpha[:P]
    self.alpha_nS = self.alpha[P:]

    self.distances_S = self.distances(X_s, X_s.shape[0], P, k)
    self.distances_nS = self.distances(X_ns, X_ns.shape[0], P, k)

    self.M_nk_S = self.M_nk(distances_S, X_s.shape[0], k)
    self.M_nk_nS = self.M_nk(distances_nS, X_ns.shape[0], k)

    self.M_k_S = self.M_k(M_nk_S, X_s.shape[0], k)
    self.M_k_nS = self.M_k(M_nk_nS, X_ns.shape[0], k)


  def x_n_hat(X, M_nk, v, N, P, k):
    x_n_hat = np.zeros((N, P))
    L_x = 0.0
    for i in range(N):
        for p in range(P):
            for j in range(k):
                x_n_hat[i, p] += M_nk[i, j] * v[j, p]
            L_x += (X[i, p] - x_n_hat[i, p]) * (X[i, p] - x_n_hat[i, p])
    return x_n_hat, L_x
  
  def yhat(M_nk, y, w, N, k):
    yhat = np.zeros(N)
    L_y = 0.0
    for i in range(N):
        for j in range(k):
            yhat[i] += M_nk[i, j] * w[j]
        yhat[i] = 1e-6 if yhat[i] <= 0 else yhat[i]
        yhat[i] = 0.999 if yhat[i] >= 1 else yhat[i]
        L_y += -1 * y[i] * np.log(yhat[i]) - (1.0 - y[i]) * np.log(1.0 - yhat[i])
    return yhat, L_y

  def M_nk(self, dists, N, k):
    M_nk = torch.zeros(N, k)
    exp = torch.zeros(N, k)
    denom = torch.zeros(N)
    for i in range(N):
        for j in range(k):
            exp[i, j] = np.exp(-1 * dists[i, j])
            denom[i] += exp[i, j]
        for j in range(k):
            if denom[i]:
                M_nk[i, j] = exp[i, j] / denom[i]
            else:
                M_nk[i, j] = exp[i, j] / 1e-6
    return M_nk
   
  def M_k(self, M_nk, N, k):
    M_k = torch.zeros(k)
    for j in range(k):
        for i in range(N):
            M_k[j] += M_nk[i, j]
        M_k[j] /= N
    return 
  
  def recover_Z():
    #TODO: fill in whatever we need to calculate LZ
    pass

  def forward(self,X,y):
    #TODO: make sure this works accordingly! do we need to readjust our parameters? do we need to call self?
    x_n_hat, L_x = x_n_hat(X, M_nk, v, N, P, k)
    y_hat, L_y = yhat(M_nk, y, w, N, k)
    Z = recover_Z()
    return x_n_hat, L_x, y_hat, L_y, Z

  def distances(self, X, N, P, k):
    dists = torch.zeros(N, P)
    for i in range(N):
        for p in range(P):
            for j in range(k):    
                dists[i, j] += (X[i, p] - self.v[j, p]) * (X[i, p] - self.v[j, p]) * self.alpha[p]
    return dists
  
  def loss_fn(self, X, y, x_n_hat, L_x, y_hat, L_y, Z):
    #TODO: calculate L = Lx + Ly + Lz
    L_z = 0.0
    for j in range(k):
        L_z += abs(M_k_sensitive[j] - M_k_nonsensitive[j])

    x_n_hat_sensitive, L_x1 = x_n_hat(data_sensitive, M_nk_sensitive, v, Ns, P, k)
    x_n_hat_nonsensitive, L_x2 = x_n_hat(data_nonsensitive, M_nk_nonsensitive, v, Nns, P, k)
    L_x = L_x1 + L_x2

    yhat_sensitive, L_y1 = yhat(M_nk_sensitive, y_sensitive, w, Ns, k)
    yhat_nonsensitive, L_y2 = yhat(M_nk_nonsensitive, y_nonsensitive, w, Nns, k)
    L_y = L_y1 + L_y2

    criterion = A_x * L_x + A_y * L_y + A_z * L_z
    return criterion
  
  def update_M_nk():
    #TODO: update M_nk after SGD
    pass
  
  def update_Mk():
    #TODO: update M_k after SGD
    pass

  def update_distances():
    #TODO: update distances
    pass


In [ ]:
model = LFR()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        x_n_hat, L_x, y_hat, L_y, Z = model.forward(X)
        loss = model.loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.update_Mnk()
        model.update_Mk()
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")